In [56]:
### Dependencies and Setup
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy import stats
from scipy.stats import linregress
import gmaps

import json
import pprint as pprint

#import api key
from api_keys import gmaps_api_key

In [57]:
#read in the csv file from WeatherPy for city data
location_data = pd.read_csv('../WeatherPy/output_data/cities.csv')
location_data.head()

,city,country,id,lat,lng,temp,humidity,clouds,wind_speed
0,Hithadhoo,MV,1282256,-0.60,73.08,83.28,78,79,14.74
1,Avarua,CK,4035715,-21.21,-159.78,84.20,83,80,9.17
2,Klaksvik,FO,2618795,62.23,-6.59,44.60,100,100,23.04
3,Hilo,US,5855927,19.73,-155.09,72.27,69,90,9.17
4,Bandarbeyla,SO,64814,9.49,50.81,79.20,79,100,19.95


In [58]:
gmaps.configure(api_key=gmaps_api_key)

In [59]:
#create heat map for humidity
#store lat and lng into locations
locations = location_data[['lat', 'lng']].astype(float)
#convert humidity to float and store
humidity = location_data['humidity'].astype(float)

In [60]:
#create humidity heatmap layer
fig = gmaps.figure(zoom_level=2, center=(0, 0))
humidity_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)
fig.add_layer(humidity_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [61]:
#locate row based on my ideal climate for vacation
ideal_spot = location_data.loc[(location_data['temp'] >= 70) & (location_data['temp'] <= 80)]
ideal_spot_2 = ideal_spot.loc[ideal_spot['wind_speed'] <= 15]
ideal_spot_3 = ideal_spot_2.loc[ideal_spot_2['clouds'] <= 35]
ideal_vacation.to_csv('output_data/ideal_vacation.csv', index=False)

In [62]:
#skip SettingWithCopyWarning, no writes back to the original frame
pd.options.mode.chained_assignment = None 

#add column for hotel name
ideal_vacation['hotel_name'] = ''
ideal_vacation['hotel_address'] = ''
ideal_vacation['hotel_rating'] = ''
ideal_vacation

,city,country,id,lat,lng,temp,humidity,clouds,wind_speed,hotel_name,hotel_address,hotel_rating
38,Puerto Ayora,EC,3652764,-0.74,-90.35,77.00,57,23,10.31,,,
171,Rocha,UY,3440777,-34.48,-54.33,70.83,49,0,7.90,,,
172,Urubicha,BO,3901969,-15.38,-62.95,77.00,55,31,1.50,,,
215,Alibag,IN,1279027,18.64,72.88,75.20,61,20,4.70,,,
242,Port Lincoln,AU,2063036,-34.73,135.87,75.43,48,0,3.85,,,
366,Kahului,US,5847411,20.89,-156.47,79.25,61,20,11.41,,,
501,Moron,AR,3430545,-34.65,-58.62,78.93,54,0,3.36,,,
525,Itaqui,BR,3460629,-29.13,-56.55,70.07,53,0,3.38,,,


In [65]:
#dictionary to update each iteration
params = {
    'radius': 5000,
    'types': 'hotel',
    'keyword': 'hotel',
    'key': gmaps_api_key
}

#identify hotel using lat/lng
for index, row in ideal_vacation.iterrows():
    lat = row['lat']
    lng = row['lng']
    
    #change location while leaving original parameters in place
    params['location'] = f'{lat},{lng}'
    
    #
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    #make request and print URL
    name_address = requests.get(base_url, params=params)
    
    #convert to json
    name_address = name_address.json()
    
    try:
        ideal_vacation.loc[index, 'hotel_name'] = name_address['results'][0]['name']
        ideal_vacation.loc[index, 'hotel_address'] = name_address['results'][0]['vicinity']
        ideal_vacation.loc[index, 'hotel_rating'] = name_address['results'][0]['rating']
    except (KeyError, IndexError):
        print('Missing field/result... skipping.')

Missing field/result... skipping.
Missing field/result... skipping.


In [66]:
ideal_vacation

,city,country,id,lat,lng,temp,humidity,clouds,wind_speed,hotel_name,hotel_address,hotel_rating
38,Puerto Ayora,EC,3652764,-0.74,-90.35,77.00,57,23,10.31,Galapagos Paradise,"Avenida Seymour S/N Fragata Esquina, Santa Cruz",3.9
171,Rocha,UY,3440777,-34.48,-54.33,70.83,49,0,7.90,,,
172,Urubicha,BO,3901969,-15.38,-62.95,77.00,55,31,1.50,,,
215,Alibag,IN,1279027,18.64,72.88,75.20,61,20,4.70,"Radisson Blu Resort & Spa - Alibaug, India","Gondhalpada, Varasoli, Alibag",4.5
242,Port Lincoln,AU,2063036,-34.73,135.87,75.43,48,0,3.85,Port Lincoln Hotel,"1 Lincoln Hwy, Port Lincoln",4.3
366,Kahului,US,5847411,20.89,-156.47,79.25,61,20,11.41,Courtyard by Marriott Maui Kahului Airport,"532 Keolani Pl, Kahului",4.3
501,Moron,AR,3430545,-34.65,-58.62,78.93,54,0,3.36,The Palm House Hotel,"Rio Negro 415 Esquina, Gral. Justo José de Urq...",4.8
525,Itaqui,BR,3460629,-29.13,-56.55,70.07,53,0,3.38,Hotel Linck,"R. Euclídes Aranha, 756 - Centro, Itaqui",4.1


In [67]:
hotel_location = ideal_vacation['hotel_name'].tolist()
hotel_location

['Galapagos Paradise',
 '',
 '',
 'Radisson Blu Resort & Spa - Alibaug, India',
 'Port Lincoln Hotel',
 'Courtyard by Marriott Maui Kahului Airport',
 'The Palm House Hotel',
 'Hotel Linck']

In [68]:
hotel_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)',
                                stroke_color='rgba(0, 150, 0, 0.4)', scale=2,
                                 info_box_content=[f'Hotel Name: {hotel}' for hotel in hotel_location]
                                )

fig = gmaps.figure()
fig.add_lyer(hotel_layer)

fig

IndexError: list index out of range